In [3]:
# set you path
import os
os.chdir("poi_medicine")
print(f'current directory: {os.getcwd()}')

import pandas as pd
import numpy as np
import geopandas as gpd

current directory: /Users/tomokitakata/Desktop/research/dev/medical_muldistribution/poi_medicine


In [5]:
uniq_dpt = pd.read_excel('A_Unique_departments_unclassi/2010.xlsx').iloc[:,1]
uniq_dpt = uniq_dpt.to_list()

ベースの分類は東大病院の診療科構成を参考。  
- 腎臓・人工透析内科：これはこれで一つあるっぽい
- ペインクリニック：内科、麻酔科、整形外科に分類される場合がある。
- 平成 20 年 4 月 1 日以降、広告することが認められない診療科名「神経科」、「呼吸器科」、「消化器科」、「胃腸科」、「循環器科」、「皮膚泌尿器科」、「性病科」、「こう門科」、「気管食道科」

In [6]:
# 分類辞書
classification_dict = {
    # 内科系
    "総合内科": [
        {"keywords": [("内", "partial")], "condition": "OR"}
    ],
    "呼吸器内科": [
        {"keywords": [("呼吸", "partial"), ("気管食道科", "partial")], "condition": "OR"}
    ],
     "消化器内科": [
        # ルール1：「内」 AND 「消化器」
        {
            "keywords": [
                ("内", "partial"),
                ("消化", "partial")
            ],
            "condition": "AND"
        },
        # ルール2：「内科」 AND 「消化器」
        {
            "keywords": [
                ("内科", "partial"),
                ("消化器", "partial")
            ],
            "condition": "AND"
        },
        # ルール3：「内」 AND 「胃」
        {
            "keywords": [
                ("内", "partial"),
                ("胃", "partial")
            ],
            "condition": "AND"
        },
        # ルール4：「内科」 AND 「胃」
        {
            "keywords": [
                ("内科", "partial"),
                ("胃", "partial")
            ],
            "condition": "AND"
        },
    ],
    "循環器内科": [
        {"keywords": [("循", "exact")], "condition": "AND NOT", "exclude": [("外科", "partial")]},
        {"keywords": [("心臓", "partial")], "condition": "AND NOT", "exclude": [("外科", "partial")]}
    ],
    "腎臓・内分泌内科": [
        {"keywords": [("腎", "partial"), ("透析", "partial"), ("人透", "exact"), ("内", "partial")], "condition": "OR"}
    ],
    "糖尿病・代謝内科": [
        {"keywords": [("糖尿", "partial"), ("代謝", "partial")], "condition": "OR"}
    ],
    "血液・腫瘍内科": [
        {"keywords": [("血液", "partial"), ("腫瘍", "partial")], "condition": "OR"}
    ],
    "アレルギー科": [
        {"keywords": [("アレ", "exact"), ("ｱﾚ", "exact")], "condition": "OR"}
    ],
    "リウマチ科": [
        {"keywords": [("リウマチ科", "exact"), ("膠原", "partial")], "condition": "OR"}
    ],
    "感染症内科": [
        {"keywords": [("感染", "partial")], "condition": "OR"}
    ],
    "脳神経内科": [
        {"keywords": [("脳", "partial"), ("神経", "partial")], "condition": "AND"}
    ],
    "老年病内科": [
        {"keywords": [("老", "partial")], "condition": "OR"}
    ],
    "心療内科": [
        {"keywords": [("心療", "exact")], "condition": "OR"}
    ],

    # 外科系
    "総合外科": [
        {"keywords": [("外科", "exact")], "condition": "AND"}
    ],
    "胃・腸外科": [
        {"keywords": [("胃", "partial"), ("腸", "partial")], "condition": "AND"}
    ],
    "大腸・肛門外科": [
        {"keywords": [("肛門", "exact"), ("こう門", "partial")], "condition": "AND"}
    ],
    "肝・胆・膵外科": [
        {"keywords": [("肝", "partial"), ("胆", "partial"), ("膵", "partial")], "condition": "AND"}
    ],
    "血管外科": [
        {"keywords": [("血", "partial")], "condition": "OR"}
    ],
    "乳腺・内分泌外科": [
        {"keywords": [("乳腺", "partial")], "condition": "OR"}
    ],
    "人工臓器・移植外科": [
        {"keywords": [("臓器", "partial"), ("移植", "partial")], "condition": "OR"}
    ],
    "心臓外科": [
        {"keywords": [("心臓", "exact"), ("外科", "exact")], "condition": "AND"}
    ],
    "呼吸器外科": [
        {"keywords": [("呼吸", "exact")], "condition": "OR"}
    ],
    "脳神経外科": [
        {"keywords": [("脳", "partial"), ("神経", "partial")], "condition": "AND"}
    ],

    # その他
    "麻酔科": [
        {"keywords": [("麻酔科", "exact"), ("疼痛", "partial")], "condition": "OR"}
    ],
    "泌尿器科": [
        {"keywords": [("泌尿器科", "exact"), ("泌", "partial"), ("尿", "partial")], "condition": "OR"}
    ],
    "皮膚科": [
        {"keywords": [("皮膚科", "exact"), ("皮", "partial")], "condition": "OR"}
    ],
    "眼科": [
        {"keywords": [("眼科", "exact"), ("コンタクト", "partial")], "condition": "OR"}
    ],
    "整形外科": [
        {"keywords": [("整形外科", "exact"), ("整", "partial")], "condition": "OR"}
    ],
    "耳鼻咽喉科・頭頸部外科": [
        {"keywords": [("耳鼻", "partial"), ("咽喉", "partial")], "condition": "OR"}
    ],
    "リハビリテーション科": [
        {"keywords": [("リハビリテーション科", "exact")], "condition": "OR"}
    ],
    "形成外科・美容外科": [
        {"keywords": [("形", "partial"), ("美", "partial")], "condition": "OR"}
    ],
    "口腔顎顔面外科・歯科": [
        {"keywords": [("歯", "partial"), ("口腔", "partial")], "condition": "OR"}
    ],
    "小児科": [
        {"keywords": [("小児科", "exact"), ("小", "partial")], "condition": "OR"}
    ],
    "小児外科": [
        {"keywords": [("小児外科", "exact")], "condition": "OR"}
    ],
    "産婦人科": [
        {"keywords": [("産婦人科", "exact")], "condition": "OR"}
    ],

    # どれにも当てはまらなかった場合
    "分類不可": []
}

def match_word(keyword: str, pattern: str, match_type: str) -> bool:
    """
    与えられた keyword(診療科名など) に対して、
    pattern が部分一致 or 完全一致するか判定する。
    """
    if match_type == "partial":
        return pattern in keyword
    elif match_type == "exact":
        return pattern == keyword
    return False

def check_rule(keyword: str, rule: dict) -> bool:
    """
    1つの rule を満たすかどうか判定する。
    """
    condition = rule.get("condition", "OR")
    rule_keywords = rule.get("keywords", [])
    exclude_keywords = rule.get("exclude", [])

    if condition == "AND":
        # rule_keywordsすべてが一致
        return all(match_word(keyword, kw, match_type) for kw, match_type in rule_keywords)

    elif condition == "OR":
        # rule_keywordsのいずれかが一致
        return any(match_word(keyword, kw, match_type) for kw, match_type in rule_keywords)

    elif condition == "AND NOT":
        # rule_keywordsすべてが一致 かつ exclude_keywordsはいずれも一致しない
        return (
            all(match_word(keyword, kw, match_type) for kw, match_type in rule_keywords)
            and not any(match_word(keyword, ex_kw, ex_match_type) for ex_kw, ex_match_type in exclude_keywords)
        )

    # それ以外の条件定義があれば拡張
    return False

def classify_keywords(keywords, classification_dict, multiple: bool = False):
    """
    与えられたキーワードリストを、分類辞書(classification_dict)に従って分類する。
    multiple=True の場合、1つのキーワードが複数の診療科に該当してもすべてに分類する。
    multiple=False の場合、最初にマッチした診療科だけに分類して終了する（デフォルト）。
    """
    # 各カテゴリに対応する空リストを用意
    classified = {category: [] for category in classification_dict.keys()}

    for keyword in keywords:
        if not isinstance(keyword, str):
            # 文字列以外は"分類不可"
            classified["分類不可"].append(keyword)
            continue

        matched_categories = []

        # 各診療科について判定
        for category, rules in classification_dict.items():
            # "分類不可" は分類対象外なのでスキップ
            if category == "分類不可":
                continue

            # ruleごとにチェックして、いずれか1つでもTrueになればその診療科に分類
            if any(check_rule(keyword, rule) for rule in rules):
                matched_categories.append(category)
                # multiple=Falseなら最初の診療科だけに分類して打ち切り
                if not multiple:
                    break

        # 1つ以上の診療科にマッチしたらそこに分類、なければ"分類不可"
        if matched_categories:
            for cat in matched_categories:
                classified[cat].append(keyword)
        else:
            classified["分類不可"].append(keyword)

    return classified

# --- 以下、サンプル実行例 ---

# キーワード例
uniq_dpt = [
    "内科", "呼吸器内科", "呼吸器外科", "心臓外科", "心臓内科", "総合外科",
    "胃腸科", "消化器内科", "整形外科", "小児科", "小児外科",
    "感染症内科", "耳鼻咽喉科", "麻酔科", "膠原病", "循環器外科", "透析科",
    "臓器移植科", "産婦人科", "皮膚科", "腎", "リウマチ科", "アレ",
    "何か適当な文字列", 1234  # 数字など文字列以外が混ざっていた例
]

classified_keywords = classify_keywords(uniq_dpt, classification_dict, multiple=False)

# 分類結果をデータフレームに整形
max_length = max(len(values) for values in classified_keywords.values())  # 最大行数を計算
data = {
    cat: values + [""] * (max_length - len(values))
    for cat, values in classified_keywords.items()
}
result_df = pd.DataFrame(data)

result_df

,総合内科,呼吸器内科,消化器内科,循環器内科,腎臓・内分泌内科,糖尿病・代謝内科,血液・腫瘍内科,アレルギー科,リウマチ科,感染症内科,...,眼科,整形外科,耳鼻咽喉科・頭頸部外科,リハビリテーション科,形成外科・美容外科,口腔顎顔面外科・歯科,小児科,小児外科,産婦人科,分類不可
0,内科,呼吸器外科,,,透析科,,,アレ,膠原病,,...,,整形外科,耳鼻咽喉科,,,,小児科,,産婦人科,心臓外科
1,呼吸器内科,,,,腎,,,,リウマチ科,,...,,,,,,,小児外科,,,総合外科
2,心臓内科,,,,,,,,,,...,,,,,,,,,,循環器外科
3,消化器内科,,,,,,,,,,...,,,,,,,,,,何か適当な文字列
4,感染症内科,,,,,,,,,,...,,,,,,,,,,1234


In [ ]:
result_df['分類不可'].to_list()

['内',
 '心内',
 '外',
 '神',
 '心外',
 '神内',
 '透内',
 '乳外',
 '矯',
 '内視鏡内',
 '病',
 '漢方）',
 '腫',
 '糖',
 '緩和ケ',
 '心',
 '救',
 '気',
 '漢内',
 '糖内',
 '化学療法内',
 '緩和',
 '矯正',
 '口外',
 '矯葉',
 nan,
 '口',
 '生殖医療）',
 '化学療法',
 '休止中',
 '心内(漢方）',
 'ひ',
 '内分内',
 '緩内',
 '臨検',
 '甲外',
 '緩医内',
 '代内',
 '感内',
 '腫内',
 '糖内分内',
 '内視内',
 '気管食道外',
 'がん',
 '漢方外来',
 '喘息外来',
 '総合診療科',
 '不妊治療',
 '基本診療科',
 '気食',
 'こう',
 '外(こう)',
 '甲内',
 '漢方内',
 '胸外',
 'り',
 '食道',
 '内視鏡外',
 '内視鏡',
 'リ',
 '乳',
 '他',
 '気外',
 '神内他',
 '感',
 '糖内他',
 '女内',
 '内代内',
 '内視',
 'リ内',
 '大',
 '内内',
 '内鏡内',
 '脂代内',
 '神緩内',
 'が緩内',
 '糖緩内',
 '総合',
 '¥n神内',
 '疼緩内',
 '疼緩',
 '腫外',
 '食外',
 '腹外',
 '気管食道',
 '外（内視鏡）',
 '内視）',
 '漢',
 '気管',
 '気管内',
 '甲状腺内',
 '乳・内外',
 '往診専門',
 '緩',
 '透',
 '気内',
 '気食外',
 'その他',
 '内秘',
 '疼内',
 '矯科',
 '内視外',
 '049-283-2133（女子栄養大学事務室）',
 '内（内視鏡）',
 '人工',
 '人',
 '内分',
 '−科',
 '−ション科',
 '生殖医療',
 '内．外',
 '鏡内',
 '外．心外',
 '代',
 '漢方',
 '健康診断',
 '本来は4と23。変更',
 '届出指導済み',
 '緩ケ',
 '気道食道科',
 'こ',
 'こ内',
 'こ外',
 'がん化学療法外',
 '頭頚部外',
 '高齢者内',
 '脂内',
 'ぺ内'

In [ ]:
result_df.to_csv('test.csv', encoding='utf-8', index=True)

診療科の分類は、医療レセプトにおける厚生労働省が指定する区分を使用。
https://www.mhlw.go.jp/topics/2009/05/dl/tp0521-1a_0053.pdf